In [84]:
from PIL import Image
import os

import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import save_img, array_to_img, load_img, array_to_img

from tensorflow.keras.layers import Add, Conv2D, Input, Lambda
from tensorflow.keras.models import Model

import numpy as np

from PIL import Image

In [85]:
train_hr_directory = './../dataset/train_dataset/HR/'
train_lr_directory = './../dataset/train_dataset/LR/'

valid_hr_directory = './../dataset/valid_dataset/HR/'
valid_lr_directory = './../dataset/valid_dataset/LR/'

In [86]:
train_dest_lr_dir = './../dataset_v3/train/LR/'
train_dest_hr_dir = './../dataset_v3/train/HR/'

test_dest_lr_dir = './../dataset_v3/test/LR/'
test_dest_hr_dir = './../dataset_v3/test/HR/'

In [87]:
BATCH_SIZE=8
NUM_PARTS=2

In [88]:
train_img_lr = image_dataset_from_directory(
    train_lr_directory,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(350, 350),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='mitchellcubic',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

# interpolation='mitchellcubic',
train_img_hr = image_dataset_from_directory(
    train_hr_directory,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(700, 700),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='mitchellcubic',
    follow_links=False,
    crop_to_aspect_ratio=False,
)


Found 3250 files belonging to 1 classes.
Found 3250 files belonging to 1 classes.


In [89]:
valid_img_lr = image_dataset_from_directory(
    valid_lr_directory,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(350, 350),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='mitchellcubic',
    follow_links=False,
    crop_to_aspect_ratio=False,
)


valid_img_hr = image_dataset_from_directory(
    valid_hr_directory,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(700, 700),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='mitchellcubic',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 300 files belonging to 1 classes.
Found 300 files belonging to 1 classes.


In [90]:
def split_image_into_parts(image_gen, output_dir, BATCH_SIZE):
    
    j = 0
    
    part_size=None
    
    for next_img_batch in image_gen:
        
        if part_size == None:
            part_size = next_img_batch[0].shape[0] // NUM_PARTS
        
        i=0
        for img_array in next_img_batch:
            
            for k in range(NUM_PARTS):
                for l in range(NUM_PARTS):
                    part = img_array[k*part_size:(k+1)*part_size, l*part_size:(l+1)*part_size]
                    
                    part_img = array_to_img(part)
                    part_filename = f"img_{j}_{i}_{NUM_PARTS*l+k}.png"
                    save_img(os.path.join(output_dir, part_filename), part_img)
            i+=1
            
        j+=1


In [95]:
split_image_into_parts(valid_img_lr, test_dest_lr_dir, BATCH_SIZE)

In [ ]:
split_image_into_parts(valid_img_hr, test_dest_hr_dir, BATCH_SIZE)

In [93]:
split_image_into_parts(train_img_lr, train_dest_lr_dir, BATCH_SIZE)

In [94]:
split_image_into_parts(train_img_hr, train_dest_hr_dir, BATCH_SIZE)

In [37]:
# Load the parts of the HR image
hr_parts = [load_img(test_dest_hr_dir+f'/img_6_3_{i}.png', target_size=(350, 350), color_mode='rgb', interpolation='lanczos') for i in range(4)]

# Combine the HR parts
combined_hr_image = Image.new('RGB', (700, 700))
combined_hr_image.paste(hr_parts[0], (0, 0))
combined_hr_image.paste(hr_parts[1], (0, 350))
combined_hr_image.paste(hr_parts[2], (350, 00))
combined_hr_image.paste(hr_parts[3], (350, 350))

# Display the combined HR image
combined_hr_image.show()